---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [19]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

,0,1,2
0,Andy,Claude,0
1,Andy,Frida,20
2,Andy,Georgia,-10
3,Andy,Joan,30
4,Andy,Lee,-10
5,Andy,Pablo,-10
6,Andy,Vincent,20
7,Claude,Frida,0
8,Claude,Georgia,90
9,Claude,Joan,0


### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [10]:
def answer_one():
        
    df = pd.read_table('Employee_Movie_Choices.txt')
    
    B = nx.Graph()
    B.add_nodes_from(df['#Employee'], bipartite=0)
    B.add_nodes_from(df['Movie'], bipartite=1)
    
    B.add_edges_from([(row['#Employee'], row['Movie']) for idx, row in df.iterrows()])
    
    return B
answer_one()

[('Andy', 'Anaconda'),
 ('Andy', 'Mean Girls'),
 ('Andy', 'The Matrix'),
 ('Claude', 'Anaconda'),
 ('Claude', 'Monty Python and the Holy Grail'),
 ('Claude', 'Snakes on a Plane'),
 ('Frida', 'The Matrix'),
 ('Frida', 'The Shawshank Redemption'),
 ('Frida', 'The Social Network'),
 ('Georgia', 'Anaconda'),
 ('Georgia', 'Monty Python and the Holy Grail'),
 ('Georgia', 'Snakes on a Plane'),
 ('Joan', 'Forrest Gump'),
 ('Joan', 'Kung Fu Panda'),
 ('Joan', 'Mean Girls'),
 ('Lee', 'Forrest Gump'),
 ('Lee', 'Kung Fu Panda'),
 ('Lee', 'Mean Girls'),
 ('Pablo', 'The Dark Knight'),
 ('Pablo', 'The Matrix'),
 ('Pablo', 'The Shawshank Redemption'),
 ('Vincent', 'The Godfather'),
 ('Vincent', 'The Shawshank Redemption'),
 ('Vincent', 'The Social Network')]

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [15]:
def answer_two():
    
    B = answer_one()
    for node in B.nodes_iter():
        if nx.get_node_attributes(B, 'bipartite')[node] == 0:
            nx.set_node_attributes(B, 'type', {node: 'employee'})
        else:
            nx.set_node_attributes(B, 'type', {node: 'movie'})
    
    return B
answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [32]:
def answer_three():
        
    B = answer_two()
    P = bipartite.weighted_projected_graph(B, employees)
    
    return P
x = answer_three()

{'weight': 2}

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [33]:
def answer_four():
    
    P = answer_three()
    
    df = pd.read_table('Employee_Relationships.txt', header=None)
    df.columns = ['E1', 'E2', 'Score']
    df['Weight'] = 0
    
    for idx, row in df.iterrows():
        try:
            df['Weight'].iloc[idx] = P[row['E1']][row['E2']]['weight']
        except:
            continue
    
    return df['Score'].corr(df['Weight'])
answer_four()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0.7883962221733477